In [6]:
import pandas as pd
import numpy as np
import math

data = pd.read_csv("train.csv")

remov = {}
remov["corr_filter"] = []
remov["nan_filter"] = []
remov["desc_filter"] = []

aux = data.copy()

In [7]:
''' filter: NaN values '''

def nan_filter(data):
    
    global remov
    
    nan_attr = {}
    sum_attr = data.isnull().sum()
        
    key = 0        
    for i in range(sum_attr.size):
        
        if(sum_attr.iloc[i] > 0):
            
            nan_attr[key] = [sum_attr.index[i], sum_attr.iloc[i]]
            key = key + 1
        
    for i in range(len(nan_attr)):
                
        if(nan_attr[i][1] >= 0.75 * data[nan_attr[i][0]].value_counts(dropna=False).sum()):
            data = data.drop(columns = nan_attr[i][0])
            remov["nan_filter"].append(nan_attr[i][0])
        elif(data[nan_attr[i][0]].dtype == 'object'):
            data[nan_attr[i][0]] = data[nan_attr[i][0]].fillna(data[nan_attr[i][0]].mode().iloc[0])
        else:
            data[nan_attr[i][0]] = data[nan_attr[i][0]].fillna(data[nan_attr[i][0]].mean())
    
    return data

data = nan_filter(data)
len(data.columns)
    

77

In [8]:
''' filter: minimun correlation with target '''

def corr_filter(data, target):
    
    global remov
    
    cor = data.corr()
    cor_target = abs(cor[target])
    irrelevant_features = cor_target[cor_target < 0.6]
        
    data = data.drop(columns = irrelevant_features.index)
    remov["corr_filter"].append(irrelevant_features.index) 
    
    return data

data = corr_filter(data, "SalePrice")

len(data.columns)


OverallQual    0.790982
TotalBsmtSF    0.613581
1stFlrSF       0.605852
GrLivArea      0.708624
GarageCars     0.640409
GarageArea     0.623431
SalePrice      1.000000
Name: SalePrice, dtype: float64


46

In [9]:
''' filter: desbalanced classes '''

def desc_filter(data):
    
    global remov
    
    objects = []
    classes_attr = {}
    
    for i in range(len(data.columns)):
    
        if(data[data.iloc[0].index[i]].dtype == "object"):
            objects.append(data.iloc[0].index[i]) 
        
    for i in range(len(objects)):
        
        class_size = data[objects[i]].value_counts().size
        classes_attr[i] = []                    

        for j in range(class_size):
            
            classes_attr[i].append(data[objects[i]].value_counts()[j])
            
    for i in range(len(classes_attr)):
        
            if(sum(classes_attr[i][1:]) < classes_attr[i][0]/2.5):
                
                remov["desc_filter"].append(objects[i])                              
                data = data.drop(columns = objects[i])
                      
    return data
                
            
data = desc_filter(data)
print(len(data.columns))

22


In [10]:
print(remov['desc_filter'])

['MSZoning', 'Street', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'RoofStyle', 'RoofMatl', 'ExterCond', 'BsmtCond', 'BsmtFinType2', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']


In [148]:
data.head()

,LotShape,Neighborhood,HouseStyle,OverallQual,Exterior1st,Exterior2nd,MasVnrType,ExterQual,Foundation,BsmtQual,...,TotalBsmtSF,HeatingQC,1stFlrSF,GrLivArea,KitchenQual,GarageType,GarageFinish,GarageCars,GarageArea,SalePrice
0,Reg,CollgCr,2Story,7,VinylSd,VinylSd,BrkFace,Gd,PConc,Gd,...,856,Ex,856,1710,Gd,Attchd,RFn,2,548,208500
1,Reg,Veenker,1Story,6,MetalSd,MetalSd,None,TA,CBlock,Gd,...,1262,Ex,1262,1262,TA,Attchd,RFn,2,460,181500
2,IR1,CollgCr,2Story,7,VinylSd,VinylSd,BrkFace,Gd,PConc,Gd,...,920,Ex,920,1786,Gd,Attchd,RFn,2,608,223500
3,IR1,Crawfor,2Story,7,Wd Sdng,Wd Shng,None,TA,BrkTil,TA,...,756,Gd,961,1717,Gd,Detchd,Unf,3,642,140000
4,IR1,NoRidge,2Story,8,VinylSd,VinylSd,BrkFace,Gd,PConc,Gd,...,1145,Ex,1145,2198,Gd,Attchd,RFn,3,836,250000
